## 1. import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
from collections import Counter

In [36]:
import hanja
from hanja import hangul

In [35]:
# pip install hanja

     -------------------------------------- 120.2/120.2 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 265.0/265.0 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 297.0/297.0 KB 3.7 MB/s eta 0:00:00
  Using cached pytest_cov-3.0.0-py3-none-any.whl (20 kB)
  Using cached coveralls-3.3.1-py2.py3-none-any.whl (14 kB)
     -------------------------------------- 187.1/187.1 KB 2.9 MB/s eta 0:00:00
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 98.7/98.7 KB 5.5 MB/s eta 0:00:00
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Created wheel for hanja: filename=hanja-0.13.3-py3-none-any.whl size=125971 sha2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 2. 데이터 불러오기

In [2]:
newsdata = pd.read_csv("../data/Newsfile3.csv", encoding='euc-kr')
newsdata.tail(3)

,date,title
58439,20151019,"[특징주]제주반도체, 美·中 대규모 투자유치에 '上'"
58440,20151019,[반도체] 2016 반도체 투자 감소 예상 - 미래
58441,20151019,"제주반도체, '해외투자유치설' 조회공시 요구"


## 3. 데이터 확인

In [3]:
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
dtypes: int64(1), object(1)
memory usage: 913.3+ KB


In [4]:
# 결측치 확인
newsdata.isnull().sum()

date     0
title    0
dtype: int64

## 4. train, test dataset 생성

In [14]:
# train, test set 분리
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(newsdata)

### 4-1. train_data

In [15]:
# train_data 확인
train_data.head(2)

,date,title
31086,20190711,"""日 수출규제는 한국 반도체 산업 견제 목적…장기화 가능성"""
55800,20160610,"해성디에스 ""자동차용 반도체 부품 생산에 무게…안정적 성장 기대해볼만"""


In [16]:
# train_data shape 확인
train_data.shape

(43831, 2)

### 4-2. test_data

In [17]:
# test_data 확인
test_data.head(2)

,date,title
3587,20211205,"반도체 판 동학개미 '中 전기차' 공략...한달새 7,000억 ""사자"""
31538,20190703,대일 보복카드는 자동차 수입·반도체 수출 규제…맞불작전 고심


In [18]:
# test_data shape 확인
test_data.shape

(14611, 2)

## 5. 데이터 전처리

### 5-1. train_data

- 한글, 영어, 숫자, 한자 이외의 문자열 제거

In [19]:
train_data["title"] = train_data["title"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|一-龥 ]", "")

- 한자가 들어간 데이터 확인

In [21]:
from collections import Counter

In [23]:
train_hanja_data = train_data.copy()

In [26]:
pattern = re.compile('[一-龥]')
# 한자 리스트 확인
hanja_list = train_hanja_data["title"].str.findall(pattern).sum()
# 빈도수가 높은 한자 체크
Counter(hanja_list).most_common()[:5]

[('美', 2140), ('中', 1906), ('車', 1372), ('日', 1086), ('韓', 970)]

In [27]:
# 한자 중복값 제거 확인
new_list = []
for i in hanja_list:
    if i not in new_list:
        new_list.append(i)
print(new_list)

['日', '兆', '中', '比', '文', '英', '美', '韓', '車', '式', '發', '弗', '硏', '稅', '證', '電', '株', '市', '社', '對', '與', '種', '裝', '新', '行', '弱', '上', '企', '外', '人', '臺', '靑', '後', '月', '尹', '協', '來', '三', '佛', '說', '功', '好', '街', '華', '高', '無', '談', '勝', '委', '路', '乙', '非', '獨', '産', '論', '超', '銀', '自', '害', '戰', '益', '多', '大', '事', '萬', '現', '星', '年', '油', '化', '問', '脫', '王', '全', '經', '故', '用', '喜', '悲', '價', '石', '癌', '訪', '伊', '向', '政', '昌', '前', '號', '甲', '亞', '和', '重', '內', '强', '曰', '火', '敵', '賞', '北', '次', '純', '水', '克', '低', '有', '面', '光', '錢', '異', '風', '會', '孝', '氣', '名', '家', '訴', '神', '母', '業', '設', '延', '枯', '死', '場', '印', '下', '民', '官', '辛', '力', '反', '史', '販', '禁', '週', '報', '國', '熱', '通', '者', '崔', '州', '色', '亂', '先', '學', '飛', '告', '燈', '投', '心', '逆', '苦', '富', '材', '道', '展', '恨', '輸', '選', '朴', '順', '塞', '翁', '之', '馬', '親', '格', '災', '共', '本', '檢', '守', '城', '他', '船', '鐵', '居', '安', '思', '危', '夢', '紙']


- `한자 -> 한글로 대체`
    - 뉴스 제목에 사용되는 한자들은 의미를 축약한 한 글자로 구성된 한자가 많은 것으로 추측됨
    - 한자 -> 한글로 단순 번역하는 방법을 시도

In [39]:
train_data["title"] = train_data["title"].apply(lambda x: hanja.translate(x, "substitution"))

In [40]:
train_data.head(3)

,date,title
31086,20190711,일 수출규제는 한국 반도체 산업 견제 목적장기화 가능성
55800,20160610,해성디에스 자동차용 반도체 부품 생산에 무게안정적 성장 기대해볼만
51511,20170407,삼성전자 깜짝실적반도체 강세 지속S 출시 Q 더 기대


### 5-2. test_data

- 한글, 영어, 숫자, 한자 이외의 문자열 제거

In [20]:
test_data["title"] = test_data["title"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|一-龥 ]", "")

In [29]:
test_hanja_data = test_data.copy()

In [30]:
pattern = re.compile('[一-龥]')
# 한자 리스트 확인
hanja_list2 = test_hanja_data["title"].str.findall(pattern).sum()
# 빈도수가 높은 한자 체크
Counter(hanja_list2).most_common()[:5]

[('美', 649), ('中', 604), ('車', 449), ('日', 356), ('韓', 308)]

- 한자가 들어간 데이터 확인

In [31]:
# 한자 중복값 제거 확인
new_list = []
for i in hanja_list2:
    if i not in new_list:
        new_list.append(i)
print(new_list)

['中', '車', '美', '兆', '日', '社', '外', '無', '株', '佛', '三', '電', '比', '尹', '證', '韓', '反', '企', '發', '臺', '結', '氣', '人', '來', '號', '硏', '新', '大', '獨', '好', '弗', '烹', '西', '安', '故', '民', '官', '文', '克', '亞', '價', '前', '對', '王', '月', '英', '委', '濠', '差', '毒', '公', '戰', '益', '難', '飛', '上', '非', '加', '行', '高', '市', '女', '風', '島', '場', '協', '年', '産', '學', '信', '全', '銀', '與', '光', '稅', '式', '亂', '檢', '力', '强', '靑', '甲', '軍', '法', '者', '洪', '脫', '談', '刮', '目', '相', '水', '國', '苦', '船', '位', '超', '多', '通', '順', '北', '弱', '課']


- `한자 -> 한글로 대체`

In [41]:
test_data["title"] = test_data["title"].apply(lambda x: hanja.translate(x, "substitution"))

In [43]:
test_data.head(3)

,date,title
3587,20211205,반도체 판 동학개미 중 전기차 공략한달새 억 사자
31538,20190703,대일 보복카드는 자동차 수입반도체 수출 규제맞불작전 고심
56144,20160502,특허청 반도체 설계기업 해외판로 개척 돕는다
